In [ ]:
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
# !pip install librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/scam

/content/drive/MyDrive/Colab Notebooks/scam


In [ ]:
# !unzip -qq "data.zip"

- wav는 매 순간의 음압을 측정하여 그 수치를 저장한 형태이기 때문에 그 자체로 음악을 분석하기에 적합하지 않음 (음의 높이와 세기를 듣는것이지 순간의 음압을 듣는게 아니기 때문)
- 푸리에 변환과 같은 변환 기법을 이용하여 시간 축의 데이터를 주파수 축의 데이터로 바꿔줘야할 필요가 있음
- 푸리에 변환 대신 푸리에 변환과 유사한 Constant-Q 변환을 사용
- Constant-Q 변환은 주파수 축이 로그 단위로 변환되고, 각 주파수에 따라 해상도가 다양하게 처리되기 때문에(저주파는 저해상도, 고주파는 고해상도) 음악을 처리하는 데에 푸리에 변환보다 유리
- 주파수 대역을 저장할 리스트 audio_cqt 선언
- constant-Q 변환할 때는 변환할 오디오 데이터와 sampling rate가 필요
- 해당 데이터에서는 sampling rate가 모두 동일하므로 따로 처리가 필요하지 않음
- 여기서는 Constant-Q 변환을 사용해 오디오 데이터를 주파수 대역으로 변환
- 변환에는 앞서 준비한 데이터를 가져와 사용하며, Constant-Q 변환에는 librosa.cqt 함수를 사용
- 여기서 n_bins는 옥타브 단계 및 개수를, bins_per_octave는 한 옥타브가 가지는 단계를 의미
- 라벨에 대해선 원 핫 인코딩을 적용

In [ ]:
def get_target_data(data_dir):
    audio_path_list = []
    audio_path_list.extend(glob(os.path.join(data_dir,'*.wav')))
    label_list = [1] * len(audio_path_list)
           
    return audio_path_list, label_list


In [ ]:
import os
from glob import glob
all_audio_path, all_label = get_target_data('/content/drive/MyDrive/Colab Notebooks/scam/data/위급상황/원천데이터/sample')

In [ ]:
len(all_audio_path), len(all_label)

(1073, 1073)

In [ ]:
all_audio_path

In [ ]:
import librosa

In [ ]:
def get_normal_data(data_dir):
    normal_path_list = []
    
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        for case_name_2 in os.listdir(current_path):
            current_path_2 = os.path.join(current_path, case_name_2)
            for case_name_3 in os.listdir(current_path_2):
                current_path_3 = os.path.join(current_path_2, case_name_3)
                normal_path_list.extend(glob(os.path.join(current_path_3,'*.wav')))
    
    label_list = [0] * len(normal_path_list)
           
    return normal_path_list, label_list


In [ ]:
all_normal_path, all_normal_label = get_normal_data('/content/drive/MyDrive/Colab Notebooks/scam/data/정상/원천데이터/KrespSpeech')

In [ ]:
len(all_normal_path), len(all_normal_label)

(5463, 5463)

## 경로를 이용해서 모든 데이터 가져온 후 음성데이터에서 

주요 개념

1. 푸리에 변환

음성 신호에 푸리에 변환을 적용하면 각 진동수 성분이 그 음성에 얼마나 들어있는지 알 수 있습니다.

쉽게 설명하면 음성 신호에 저음이 얼마나 있고, 고음이 얼마나 있는지를 정량적으로 구할 수 있습니다.

2. STFT(Short Time Fourier Transform)

음성을 작게(0.01초 수준) 잘라서 각 작은 조각에 푸리에 변환을 적용할 수 있습니다.
이것을 STFT라고 부르고 일반적으로 이 결과의 L2 norm을 Spectrogram이라고 부릅니다.

3. Melspectrogram

Melspectrogram은 Spectrogram에 mel-filter라는 필터를 적용해서 얻어집니다. 이는 사람의 청각 기관이 저음에서 주파
수 변화에 민감하고 고음에서는 주파수의 변화에 덜 민감한 특징을 반영하고 있습니다.

- Function and Option

torchaudio에서는 다음과 같이 Spectrogram과 Melspectrogram을 얻을 수 있는 프로세스를 정의할 수 있습니다.
AmplitudeToDB는 power단위의 Spectrogram 또는 Melspectrogram을 dB(로그) 단위로 변환해 줍니다.

1. win_length

음성을 작은 조각으로 자를 때 작은 조각의 크기를 의미합니다.
자연어 처리 분야에서는 25ms의 크기를 기본으로 하고 있으며 16000Hz인 음성에서는 400에 해당하는 값입니다. (16000/40)
22050Hz의 파일이기 때문에 552로 설정하겠습니다. (22050/40, 소수점 올림)

2. n_fft

win_length의 크기로 잘린 음성의 작은 조각은 0으로 Padding 되어서 n_fft로 크기가 맞춰집니다. 그렇게 padding 된 조각
에 푸리에 변환이 적용됩니다. 따라서 n_fft는 win_length보다 크거나 같아야 하고 일반적으로 속도를 위해서 값으로 설정
합니다.
win_length의 크기가 552이기 때문에 512는 될 수 없고, 1024는 너무 크다고 생각하여 win_length와 똑같이 552로 설정하
겠습니다.

3. hop_length

음성을 작은 조각으로 자를 때, 자르는 간격을 의미합니다. 즉, 이 길이만큼 옆으로 밀면서 작은 조각을 얻습니다.
일반 적으로 10ms의 크기를 기본으로 하고 있으며 16000Hz인 음성에서는 160에 해당하는 값입니다. (16000/100)
22050Hz의 파일이기 때문에 221로 설정하겠습니다. (22050/100)

4. n_mels

적용할 mel filter의 개수를 의미합니다

n_mel이 80이면 Melspectrogram의 주파수 방향 성분 수는 80인 것입니다.


마지막으로 Spectrogram과 Melspectrogram의 해상력에 대해 설명하겠습니다.

win_length가 커질수록 주파수 성분에 대한 해상력은 높아지지만(정밀해진다) 시간 성분에 대한 해상력은 낮아지게 됩니다.

즉, 더 정밀한 주파수 분포를 얻을 수 있으나 시간에 따른 주파수 변화를 관찰하기가 어려워집니다.

반대로 win_length가 작은 경우에는 주파수 성분에 대한 해상력은 낮아지지만, 시간 성분에 대한 해상력은 높아지게 됩니다.

따라서 적절한 값을 찾는 것이 중요합니다.

또한 n_fft를 키우는 경우 주파수 성분의 수는 증가할지 몰라도 실제 주파수의 해상력은 증가하지 않습니다

'마지막으로 spectrogram과 melspectrogram의 해상력에 대해 설명하겠습니다. win_length가 커질수록 주파수 성분에 대한 해상력은 높아지지만, 즉 더 정밀해지지만, 시간 성분에 대한 해상력은 낮아지게 됩니다. 즉, 더 정밀한 주파수 분포를 얻을 수 있으나 시간에 따른 주파수 변화를 관찰하기가 어려워집니다. 반대로 win_length가 작은 경우에는 주파수 성분에 대한 해상력은 낮아지지만, 시간 성분에 대한 해상력은 높아지게 됩니다. 따라서 적절한 값을 찾는 것이 중요합니다.'

In [ ]:
import pandas as pd

df = []
df_log = []
target = []

In [ ]:
import librosa
import numpy as np

In [ ]:

for path in all_audio_path:
    y, sr = librosa.load(path)

    n_fft = int(np.ceil(0.025 * sr))
    win_length = int(np.ceil(0.025 * sr))
    hop_length = int(np.ceil(0.01 * sr))

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, win_length=win_length, hop_length=hop_length, n_fft=n_fft)

#     S_dB = librosa.power_to_db(S, ref=np.max)
    df.append(S)
#     df_log.append(S_dB)
    target.append(1)

 n_mels 는 에러가 나지않는 최대 값을 해야함.


In [ ]:
for path in all_normal_path:
    y, sr = librosa.load(path)

    n_fft = int(np.ceil(0.025 * sr))
    win_length = int(np.ceil(0.025 * sr))
    hop_length = int(np.ceil(0.01 * sr))

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, win_length=win_length, hop_length=hop_length, n_fft=n_fft)


#     S_dB = librosa.power_to_db(S, ref=np.max)
    df.append(S)  
#     df_log.append(S_dB)
    target.append(0)

In [ ]:
arr_df = []
arr_target = []
import itertools

for i in range(len(df)):
    if df[i].shape[0] * df[i].shape[1] < 50000:
        arr_df.append( list(itertools.chain(*df[i]))) 
        arr_target.append(target[i])

In [ ]:
max_len = max(len(item) for item in arr_df)
print('최대 길이 :',max_len)

최대 길이 : 69760


In [ ]:

for sentence in arr_df:
    while len(sentence) < max_len:
        sentence.append(0)

padded_np = np.array(arr_df)
padded_np

In [ ]:
padded_np[0].shape

In [ ]:
target_np = np.array(arr_target)
target_np.shape

(499,)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(padded_np)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_np,
    target_np,
    test_size=0.2,
    random_state=0
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)
pred = DT.predict(X_test)
acc = accuracy_score(pred, y_test)
print(acc)


0.7971014492753623


## 머신러닝 모델 중에서 성능이 가장 뛰어났던 DecisionTreeClassifier를 저장

In [ ]:
# import joblib
# joblib.dump(model, 'DecisionTree_model.pkl') 

# CNN 으로 돌리기 ( 80, 249 )

In [ ]:
padded_np.shape

(499, 19840)

In [ ]:
b = padded_np.reshape(499, 80, 248)
b.shape

(499, 80, 248)

In [ ]:
re_np = np.expand_dims(b, -1)
re_np.shape

(499, 80, 248, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    re_np,
    target_np,
    test_size=0.2,
    random_state=0
)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten

def model_CNN_build():
    model = Sequential()
    input = Input(shape=(80,248,1))

    output = Conv2D(128, 3, strides=1, padding='same', activation='relu')(input)
    output = MaxPool2D(pool_size=(2,2), strides=2, padding='same')(output)
    Dropout(0.15)
    
    output = Conv2D(256, 3, strides=1, padding='same', activation='relu')(output)
    output = MaxPool2D(pool_size=(2,2), strides=2, padding='same')(output)
    Dropout(0.15)

    output = Conv2D(512, 3, strides=1, padding='same', activation='relu')(output)
    output = MaxPool2D(pool_size=(2,2), strides=2, padding='same')(output)
    Dropout(0.15)

    output=Flatten()(output)
    Dropout(0.2),
    output=Dense(512, activation='relu')(output)
    output=Dense(256, activation='relu')(output)
    output=Dense(128, activation='relu')(output)

    output=Dense(1, activation='sigmoid')(output)
    
    model = Model(inputs=[input], outputs=output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

    return model



In [ ]:
model = model_CNN_build()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80, 248, 1)]      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 80, 248, 128)      1280      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 40, 124, 128)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 40, 124, 256)      295168    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 20, 62, 256)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 62, 512)       1180

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('CNN_best_model.h5', monitor='acc', mode='max', verbose=1, save_best_only=True)


hist = model.fit(X_train, y_train, epochs=100, verbose=0, callbacks=[es,mc], batch_size=128, validation_split=0.2)




Epoch 1: acc improved from -inf to 0.49216, saving model to CNN_best_model.h5

Epoch 2: acc improved from 0.49216 to 0.60502, saving model to CNN_best_model.h5

Epoch 3: acc improved from 0.60502 to 0.76489, saving model to CNN_best_model.h5

Epoch 4: acc did not improve from 0.76489

Epoch 5: acc improved from 0.76489 to 0.78683, saving model to CNN_best_model.h5

Epoch 6: acc did not improve from 0.78683

Epoch 7: acc improved from 0.78683 to 0.82759, saving model to CNN_best_model.h5

Epoch 8: acc improved from 0.82759 to 0.87147, saving model to CNN_best_model.h5

Epoch 9: acc did not improve from 0.87147

Epoch 10: acc improved from 0.87147 to 0.88401, saving model to CNN_best_model.h5

Epoch 11: acc improved from 0.88401 to 0.88715, saving model to CNN_best_model.h5

Epoch 12: acc improved from 0.88715 to 0.92476, saving model to CNN_best_model.h5

Epoch 13: acc improved from 0.92476 to 0.94984, saving model to CNN_best_model.h5

Epoch 14: acc improved from 0.94984 to 0.97179, s

In [ ]:
model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 42ms/step - loss: 1.2928 - acc: 0.7900


[1.2928054332733154, 0.7900000214576721]

In [ ]:
pred = model.predict(X_test)

pred = pred.round()
pred = pred.flatten() # 1차원으로 변경

from sklearn.metrics import recall_score, precision_score

recall_score(y_test,pred), precision_score(y_test,pred)

(0.9361702127659575, 0.7096774193548387)